In [1]:
!pip install langchain
!pip install accelerate
!pip install sentence-transformers
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00


In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from accelerate import *

In [4]:
from zipfile import ZipFile
with ZipFile('/content/txts1.zip', 'r') as zip_ref:
    zip_ref.extractall('txts1')

In [5]:
# Check extracted contents
import os
os.listdir('txts1/txts1')

['THE PROTECTION OF WOMEN FROM DOMESTIC VIOLENCE ACT, 2005.txt',
 'THE RECOVERY OF DEBTS AND BANKRUPTCY ACT, 1993.txt',
 'THE SCHEDULED CASTES AND THE SCHEDULED TRIBES.txt',
 'THE LEGAL SERVICES AUTHORITIES ACT, 1987 .txt',
 'THE CENTRAL EXCISE ACT, 1944.txt',
 'THE SECURITISATION AND RECONSTRUCTION OF FINANCIAL ASSETS AND ENFORCEMENT OF SECURITY INTEREST ACT, 2002.txt',
 'THE IMMORAL TRAFFIC (PREVENTION) ACT, 1956.txt',
 'THE DELHI MUNICIPAL CORPORATION ACT, 1957.txt',
 'THE FACTORIES ACT, 1948.txt',
 'THE CODE ON SOCIAL SECURITY ACT, 2020.txt',
 'THE RESERVE BANK OF INDIA ACT, 1934.txt',
 'THE INDIAN CONTRACT ACT, 1872.txt',
 'THE BHARATIYA NYAYA SANHITA, 2023.txt',
 'THE NATIONAL BANK FOR FINANCING INFRASTRUCTURE AND DEVELOPMENT ACT, 2021.txt',
 'THE WATER (PREVENTION AND CONTROL OF POLLUTION) ACT, 1974.txt',
 'THE ARMS ACT, 1959.txt',
 'THE MINIMUM WAGES ACT, 1948.txt',
 'THE INDIAN PARTNERSHIP ACT, 1932.txt',
 'THE INFORMATION TECHNOLOGY ACT, 2000.txt',
 'The-Trade-Unions-Act-1926

In [6]:
folder_path = r"txts1/txts1"
documents = []

# Read all .txt files and chunk them
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):
        with open(os.path.join(folder_path, file_name), "r", encoding="utf-8") as f:
            raw_text = f.read()
            chunks = splitter.split_text(raw_text)
            documents.extend(chunks)

In [7]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-7-83794808db26>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 80.3 MB/s eta 0:00:00


In [9]:
vectorstore = FAISS.from_texts(documents, embedding=embeddings)
vectorstore.save_local("legal_vector_index")

In [10]:
# Load your vector store
vectorstore = FAISS.from_texts(documents, embedding=embeddings)

In [11]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `mistral-access` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate 

In [12]:
# Load Mistral 7B (make sure it's downloaded or use a local server like ollama or text-gen-webui)
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [13]:
# Set up LLM pipeline
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

Device set to use cuda:0
<ipython-input-13-dd562ae8b173>:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [14]:
# Build QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

In [15]:
# Ask a question
query = input("Enter your question: ")
query = query.lower()
response = qa_chain(query)
print(response["result"])

Enter your question: Mohan owned agricultural land in a rural area, but his neighbor, Suresh, began constructing a building on a portion of Mohan’s land without permission. Mohan discovered the encroachment too late and sought legal action under property law to reclaim his land and prevent further construction.  Legal Issues Involved:  Rights of landowners under the Transfer of Property Act, 1882  Remedies under the Specific Relief Act, 1963  Encroachment prevention under local municipal laws  Training Question: How would you assist Mohan in filing a legal case against Suresh for land encroachment? What legal remedies could be pursued under Indian law?


<ipython-input-15-1c65c11e0b91>:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain(query)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

"3-B. Conditions for entertainment of suits.— (1) No suit or proceeding for. — 
(a) declaration of title or any right over any agricultural land, with or without any other relief; o r
(b) specific performance of any contract for transfer of any agricultural land, with or without any other relief, shall be 
entertained by any Court, unless the plaintiff or applicant, as the case may be, knowing or having reason to believe that 
a return under section 9 of the Madhya Pradesh Ceiling on Agricultural Holdings Act, 1960 (No. 20 of 1960) in relation to land aforesaid has been or is required to be filed by him or by any other person before competent authority appointed 
43

"3-B. Conditions for entertainment of suits.— (1) No suit or proceeding for. — 
(a) declaration of title or any right over any agricultural land, with or withou

In [16]:

query = "Consider that you are a legal assistant, now evaluate: " + input("Enter your question: ")
query = query.lower()
response = qa_chain(query)
print(response["result"])

Enter your question: Mohan owned agricultural land in a rural area, but his neighbor, Suresh, began constructing a building on a portion of Mohan’s land without permission. Mohan discovered the encroachment too late and sought legal action under property law to reclaim his land and prevent further construction.  Legal Issues Involved:  Rights of landowners under the Transfer of Property Act, 1882  Remedies under the Specific Relief Act, 1963  Encroachment prevention under local municipal laws  Training Question: How would you assist Mohan in filing a legal case against Suresh for land encroachment? What legal remedies could be pursued under Indian law?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

"3-B. Conditions for entertainment of suits.— (1) No suit or proceeding for. — 
(a) declaration of title or any right over any agricultural land, with or without any other relief; o r
(b) specific performance of any contract for transfer of any agricultural land, with or without any other relief, shall be 
entertained by any Court, unless the plaintiff or applicant, as the case may be, knowing or having reason to believe that 
a return under section 9 of the Madhya Pradesh Ceiling on Agricultural Holdings Act, 1960 (No. 20 of 1960) in relation to land aforesaid has been or is required to be filed by him or by any other person before competent authority appointed 
43

"3-B. Conditions for entertainment of suits.— (1) No suit or proceeding for. — 
(a) declaration of title or any right over any agricultural land, with or withou

In [17]:
model.save_pretrained("./my_model")
tokenizer.save_pretrained("./my_model")

('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/tokenizer.model',
 './my_model/added_tokens.json',
 './my_model/tokenizer.json')

In [18]:
!pip install gradio transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00


In [19]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from safetensors.torch import load_file
import gradio as gr
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from safetensors.torch import load_file
import torch

In [20]:
model_path = "/content/my_model"  # Update with your actual folder path
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [21]:
# Load SafeTensors model manually
state_dict = load_file(f"{model_path}/model-00006-of-00006.safetensors")
model = AutoModelForCausalLM.from_pretrained(model_path,use_safetensors=True)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [22]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [23]:
demo = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs="text",
    title="LLM Demo",
    description="Enter a prompt and get a response from your trained LLM model."
)

In [24]:
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6659041ca0b11a2265.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
